In [ ]:
! pip install tqdm boto3 requests regex sentencepiece sacremoses
! pip install transformers
! pip install sentence_transformers
! pip install -U sentence-transformers
# ! pip install numpy
! pip install torch
! pip install torchtext
! pip install torchmetrics
! pip install pytorch-lightning
! pip install time
! pip install ipykernel
! pip install spacy
! pip install "grpcio>=1.37.0,<2.0" "h5py>=3.6.0,<3.7" "numpy>=1.22.3,<1.23.0"

In [ ]:
import json
import pandas as pd
import collections
import torch
import torch.nn as nn
import numpy as np
import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import random
import re
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import math
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class BERT_Data:
    '''
    Class that cleans and formats the bills and news datasets for the BERT
    model, tokenizes the data, and creates then saves the text embeddings.
    '''
    def __init__(self, random_seed = 5,
                bert_model = 'bert-base-uncased', #'bert-base-uncased' or 'bert-large-uncased'
                date_range_begin = None, date_range_end = '2018-04-01',
                bills_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/bills_data/115th.csv',
                clean_bills_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/bills_data/115th_clean.csv',
                minimal_clean_bills_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/bills_data/115th_clean_minimal.csv',
                foxnews_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/fox.csv',
                clean_foxnews_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/fox_clean.csv',
                minimal_clean_foxnews_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/fox_clean_minimal.csv',
                breitbart_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/breitbart.csv',
                clean_breitbart_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/breitbart_clean.csv',
                minimal_clean_breitbart_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/breitbart_clean_minimal.csv',
                cnn_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/cnn.csv',
                clean_cnn_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/cnn_clean.csv',
                minimal_clean_cnn_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/cnn_clean_minimal.csv',
                nytimes_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/nyt.csv',
                clean_nytimes_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/nyt_clean.csv',
                minimal_clean_nytimes_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/nyt_clean_minimal.csv',
                truncated_minimal_clean_nytimes_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/nyt_clean_minimal_truncated.csv',
                wapo_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/washington_post_with_date.csv',
                clean_wapo_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/washington_post_with_date_clean.csv',
                minimal_clean_wapo_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/washington_post_with_date_clean_minimal.csv',
                truncated_minimal_clean_wapo_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/washington_post_with_date_clean_minimal_truncated.csv'
    ):
        self.device = self.cuda_mps_cpu()
        self.random_seed = random_seed
        self.random_seed_function()
        self.date_range_begin = date_range_begin
        self.date_range_end = date_range_end
        self.bills_csvpath = bills_csvpath
        self.clean_bills_csvpath = clean_bills_csvpath
        self.minimal_clean_bills_csvpath = minimal_clean_bills_csvpath
        self.df_bills_prepared = pd.read_csv(minimal_clean_bills_csvpath)
        # self.df_bills_raw = pd.read_csv(bills_csvpath)
        # self.df_bills_clean = self.clean_bills()
        # self.df_bills_clean = pd.read_csv(clean_bills_csvpath)
        self.foxnews_csvpath = foxnews_csvpath
        self.minimal_clean_foxnews_csvpath = minimal_clean_foxnews_csvpath
        self.breitbart_csvpath = breitbart_csvpath
        self.minimal_clean_breitbart_csvpath = minimal_clean_breitbart_csvpath
        self.cnn_csvpath = cnn_csvpath
        self.minimal_clean_cnn_csvpath = minimal_clean_cnn_csvpath
        self.nytimes_csvpath = nytimes_csvpath
        self.minimal_clean_nytimes_csvpath = minimal_clean_nytimes_csvpath
        self.truncated_minimal_clean_nytimes_csvpath = truncated_minimal_clean_nytimes_csvpath
        self.wapo_csvpath = wapo_csvpath
        self.minimal_clean_wapo_csvpath = minimal_clean_wapo_csvpath
        self.truncated_minimal_clean_wapo_csvpath = truncated_minimal_clean_wapo_csvpath
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model)
        self.bert_base = AutoModel.from_pretrained(bert_model)
        self.practice_bill = self.df_bills_prepared.head(5).copy()


    def clean_bills(self, only_2017 = False, only_2018 = False,
                save = True # only_bills = False,
    ):
        '''
        Clean and format the bills dataset
        '''
        df = pd.read_csv(self.bills_csvpath)
        df.loc[:, ['new_index']] = df.index
        df.loc[:, 'cleaned_text'] = df.loc[:, 'raw_text'].apply(
                self.clean_bill_text, args=()
        )
        df.loc[:, ['date']] = pd.to_datetime(
                    df.loc[:, 'introduced_date'], format='%Y-%m-%d'
        )
        df.loc[:, ['house_passage_binary']] = df.loc[:, 'house_passage'].fillna(0, inplace=True)

        df.loc[:, ['house_passage_binary']] = np.where(
                df.loc[:, 'house_passage_binary'] != 0, 1, 0
        )

        if only_2017:
            df = df.loc[(df.loc[:, 'date'] >= '2017-01-01'
                    & df.loc[:, 'date'] < '2018-01-01'
            ), :]

        if only_2018:
            df = df.loc[(df.loc[:, 'date'] >= '2018-01-01'
                    & df.loc[:, 'date'] < '2019-01-01'
            ), :]

        df.loc[:, ['cleaned_text']] = df.loc[:, 'cleaned_text'].apply(
                self.clean_generalnews_text, args=()
        )
        df = df.loc[:, ['bill_id', 'new_index', 'cleaned_text', 'date', 'house_passage_binary', 'bill_type']]

        if save:
            df.to_csv(self.clean_bills_csvpath, index=False)
        # if only_bills:
        #     df = df.loc[(df.loc[:, 'bill_type'] == | df.loc[:, 'bill_type'] == ), :]

        return df


    def dates_clean_news(self, df_whole, Date_version, long_date_version,
            start_date = None, end_date = None,
            minimal_columns = False
            # minimal_columns = ['index', 'date', 'cleaned_text']
    ):
        '''
        Format the date of the news articles to match the date of the bills
        '''
        if start_date is None:
            start_date = self.date_range_begin
        if end_date is None:
            end_date = self.date_range_end

        df = df_whole.copy()
        if Date_version:
            df.loc[:, ['date']] = pd.to_datetime(
                    df.loc[:, 'Date'], format='%Y-%m-%d'
            )
        elif long_date_version:
            df.loc[:, ['date']] = pd.to_datetime(
                    df.loc[:, 'date'].str[:10], format='%Y-%m-%d'
            )
        else:
            df.loc[:, ['date']] = pd.to_datetime(
                    df.loc[:, 'date'], format='%Y-%m-%d'
            )

        if start_date is not None:
            df = df.loc[df.loc[:, 'date'] >= start_date, :]
        if end_date is not None:
            df = df.loc[df.loc[:, 'date'] <= end_date, :]

        if minimal_columns is not False:
            df = df.loc[:, minimal_columns]

        return df


    def clean_foxnews(self, save = True):
        '''
        Clean and format the Fox News data
        '''
        df_fox = pd.read_csv(self.foxnews_csvpath)
        df_fox.loc[:, ['new_index']] = df_fox.index
        df_fox_dated = self.dates_clean_news(df_fox, Date_version = True,
                long_date_version = False
        )
        df_fox_dated.loc[:, 'cleaned_text'] = df_fox_dated.loc[:, 'article_text'].apply(
                self.clean_foxnews_text, args=()
        )
        df_fox_dated = df_fox_dated.loc[:, ['uuid', 'new_index', 'cleaned_text', 'date']]

        if save:
            df_fox_dated.to_csv(self.minimal_clean_foxnews_csvpath, index=False)

        return df_fox_dated


    def clean_breitbart(self, save = True):
        '''
        Clean and format the Breitbart data
        '''
        df_breitbart = pd.read_csv(self.breitbart_csvpath)
        df_breitbart.loc[:, ['new_index']] = df_breitbart.index
        df_breitbart_dated = self.dates_clean_news(df_breitbart,
                Date_version = True, long_date_version = False
        )
        df_breitbart_dated.loc[:, 'cleaned_text'] = df_breitbart_dated.loc[:, 'article_text'].apply(
                self.clean_generalnews_text, args=()
        )
        df_breitbart_dated = df_breitbart_dated.loc[:, ['uuid', 'new_index', 'cleaned_text', 'date']]
        # df_breitbart_dated.replace('', np.nan, inplace=True)
        # df_breitbart_dated.dropna(subset= ['cleaned_text'], inplace=True)

        if save:
            df_breitbart_dated.to_csv(self.minimal_clean_breitbart_csvpath,
                    index=False
            )

        return df_breitbart_dated


    def clean_cnn(self, save = True):
        '''
        Clean and format the CNN data
        '''
        df_cnn = pd.read_csv(self.cnn_csvpath)
        df_cnn.loc[:, ['new_index']] = df_cnn.index
        df_cnn_dated = self.dates_clean_news(df_cnn, Date_version = False,
                long_date_version = True
        )
        df_cnn_dated.loc[:, 'cleaned_text'] = df_cnn_dated.loc[:, 'text'].apply(
                self.clean_generalnews_text, args=()
        )
        df_cnn_dated = df_cnn_dated.loc[:, ['title', 'new_index', 'cleaned_text', 'date']]

        if save:
            df_cnn_dated.to_csv(self.minimal_clean_cnn_csvpath, index=False)

        return df_cnn_dated


    def clean_nytimes(self, save = True):
        '''
        Clean and format the New York Times data
        '''
        df_nytimes = pd.read_csv(self.nytimes_csvpath)
        df_nytimes.loc[:, ['new_index']] = df_nytimes.index
        df_nytimes_dated = self.dates_clean_news(df_nytimes,
                Date_version = False, long_date_version = True
        )
        df_nytimes_dated.loc[:, 'cleaned_text'] = df_nytimes_dated.loc[:, 'text'].apply(
                self.clean_generalnews_text, args=()
        )
        df_nytimes_dated = df_nytimes_dated.loc[:, ['title', 'new_index', 'cleaned_text', 'date']]

        if save:
            df_nytimes_dated.to_csv(self.minimal_clean_nytimes_csvpath,
                    index=False
            )

        return df_nytimes_dated


    def clean_wapo(self, save = True):
        '''
        Clean and format the Washington Post data
        '''
        df_wapo = pd.read_csv(self.wapo_csvpath)
        df_wapo.loc[:, ['new_index']] = df_wapo.index
        df_wapo_dated = self.dates_clean_news(df_wapo, Date_version = False,
                long_date_version = False
        )
        df_wapo_dated.loc[:, 'cleaned_text'] = df_wapo_dated.loc[:, 'text'].apply(
                self.clean_generalnews_text, args=()
        )
        df_wapo_dated = df_wapo_dated.loc[:, ['title', 'new_index', 'cleaned_text', 'date']]

        if save:
            df_wapo_dated.to_csv(self.minimal_clean_wapo_csvpath, index=False)

        return df_wapo_dated


    def clean_bill_text(self, text, glove=None, cbows=None, lemma=None):
        '''
        Intakes the text of a single bill and removes formating unique to the
        bill text, eliminates html tags, separates sentences,
        lower cases, and correctly formats the text
        '''
        text = text.lower()
        text = re.compile('<.*?>').sub('', text)
        text = re.compile(r"[_]").sub(" ", text)
        text = re.sub(r"\[.*?\]", "", text)
        text = re.sub(r"\d+\sU\.S\.C\.\s\d+[a-z]?(\(\d+\))?( \([a-z]+\))?", "",
                text
        )
        text = re.sub(r'(sec\.\s+\d+\.?)|(section\s+\d+\.)', '', text)
        text = re.sub(r'\(\d+ u\.s\.c\. \d+\([a-z]\)(\(\d+\))*\)', '', text)
        text = re.sub(r'\(\d+\s+u\.s\.c\.\s+\d+\)', '', text)
        text = re.sub(r'\(\d+ u\.s\.c\. \d+\)', '', text)
        text = re.sub(r'\(\d+\)', '', text)
        text = re.sub(r'\([ivxlcdm]+\)', '', text)
        text = re.sub(r'\(\w\)', '', text)
        text = re.sub(r'\[\d+\]', '', text)
        text = re.sub(r'\n', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'([a-z])\.--([a-z])', r'\1. -- \2', text)
        text = text.replace('``', '"').replace("''", '"')
        text = text.strip()

        return text


    def clean_foxnews_text(self, text):
        '''
        Intakes a single article and removes formating unique to the Fox News
        text, lower cases, and correctly formating the text
        '''
        text = text.replace("Get the latest updates from the 2024 campaign trail, exclusive interviews and more Fox News politics content.SubscribedYou've successfully subscribed to this newsletter!", " ")
        text = text.lower()
        text = re.sub(r"([.!?;:\"“”])(?=[^\s])|”(?=\w)", r"\1 ", text)
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()

        return text


    def clean_generalnews_text(self, text):
        '''
        Intakes a single article and removes formating unique to the General
        News text, lower cases, separates connected end of sentences,
        and correctly formating the text
        '''
        text = re.sub(r"http\S+", " ", text)
        text = re.sub(r"@\S+", " ", text)
        text = re.sub(r'([a-z])\.”([A-Z])', r'\1. ” \2', text)
        text = text.lower()
        text = re.sub(r"([.!?;:\"“”])(?=[^\s])|”(?=\w)", r"\1 ", text)
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()

        return text


    def long_texts_dim_reduction(self, embedding_tensor,
                dim_reduction_strategy
    ): # Pooling, max, (potential: PCA)
        '''
        If tokenized text is untruncated and longer than 510 tokens, adds a
        secondary measure to reduce the dimensionality of the embedding
        tensor
        '''
        if dim_reduction_strategy == 'mean':
            embedding_tensor = embedding_tensor.mean(dim=0)
        elif dim_reduction_strategy == 'max':
            embedding_tensor = embedding_tensor.max(dim=0).values

        print(embedding_tensor.shape)
        return embedding_tensor


    def bert_embed_single_row(self, text, index = None, pooling = 'mean',
            max_len = 510, attention_mask_onpadding = True,
            long_dim_reduction_strategy = 'max',
            skip_long_texts = False, truncate_text = False,
    ): #512 tokens BERT # max, mean, CLS token embedding
        '''
        Intakes an entire bill or article, tokenizens the text, embeds the text
        using the BERT model, applies pooling, and returns the embedding tensor.
        Text may be truncated to max_len tokens.
        '''
        print(f'Bill/News index: {index}')
        if truncate_text:
            text_tokenized = self.tokenizer(text, return_tensors='pt',
                add_special_tokens=False, truncation=True, max_length = max_len
            )
        else:
            text_tokenized = self.tokenizer(text, return_tensors='pt',
                add_special_tokens=False, truncation=False
            )
        text_len = len(text_tokenized.input_ids[0])
        if skip_long_texts:
            if text_len > max_len:
                return None
        # print(text_tokenized.input_ids[0])
        text_segment_count = math.ceil(text_len / max_len)
        embedding_tensor = torch.tensor([], dtype = torch.float32)
        for segment in range(text_segment_count):
            start_token = segment * max_len
            end_token = min(start_token + max_len, text_len)
            # print(f'Segment {segment}, start_token {start_token}, end_token {end_token}')
            # print(self.tokenizer.cls_token_id, self.tokenizer.sep_token_id)
            segment_ids_tensor = text_tokenized.input_ids[0, start_token:end_token]
            padded_token_count = (max_len) - (end_token - start_token)
            if padded_token_count == 0:
                segment_ids_tensor = torch.cat([
                        torch.tensor([self.tokenizer.cls_token_id]),
                        segment_ids_tensor,
                        torch.tensor([self.tokenizer.sep_token_id])
                ]).unsqueeze(0)
            else:
                padding_tokens_tensor = torch.tensor(
                        [self.tokenizer.pad_token_id] * padded_token_count
                )
                segment_ids_tensor = torch.cat([
                        torch.tensor([self.tokenizer.cls_token_id]),
                        segment_ids_tensor,
                        padding_tokens_tensor,
                        torch.tensor([self.tokenizer.sep_token_id])
                ]).unsqueeze(0)
            if attention_mask_onpadding:
                attention_mask = (segment_ids_tensor !=
                        self.tokenizer.pad_token_id).to(dtype=torch.int64
                )
                with torch.no_grad():
                    segment_embeddings_tensor = self.bert_base(
                            input_ids = segment_ids_tensor,
                            attention_mask=attention_mask
                    ).last_hidden_state
            else:
                with torch.no_grad():
                    segment_embeddings_tensor = self.bert_base(
                            segment_ids_tensor
                    ).last_hidden_state

            if segment == 0:
                total_embeddings_tensor = segment_embeddings_tensor
            else:
                # print('total_embeddings_tensor: ', total_embeddings_tensor.shape)
                # print('segment_embeddings_tensor: ', segment_embeddings_tensor.shape)
                total_embeddings_tensor = torch.cat([total_embeddings_tensor,
                        segment_embeddings_tensor
                ])

        if pooling == 'mean':
            output_embedding_tensor = total_embeddings_tensor.mean(dim=1)
        elif pooling == 'max':
            output_embedding_tensor = total_embeddings_tensor.max(dim=1).values
        else: # None or CLS implies CLS
            output_embedding_tensor = total_embeddings_tensor[:, 0, :]

        if not skip_long_texts:
            if not truncate_text:
                output_embedding_tensor = self.long_texts_dim_reduction(
                        output_embedding_tensor, long_dim_reduction_strategy
                )

        print('output_embedding_tensor: ', output_embedding_tensor.shape)
        return output_embedding_tensor


    def total_bert_embeddings(self, df, text_column, pooling = 'mean',
            max_len = 510, attention_mask_onpadding = True,
            long_dim_reduction_strategy = 'max',
            skip_long_texts = False, truncate_text = False
    ):
        '''
        Intakes a dataframe, facilitates the tokenization and embedding process
        using the BERT model for each row in the dataframe.  Returns the
        total embedding tensor.  Text may be truncated to max_len tokens.
        '''
        embeddings_lst = []

        for index, row in df.iterrows():
            ## Used in testing
            # if index == 10:
            #     break
            embedded_row = self.bert_embed_single_row(row.loc[text_column],
                            index, pooling, max_len, attention_mask_onpadding,
                            long_dim_reduction_strategy, skip_long_texts,
                            truncate_text
            )
            # If skip_long_texts is True and text after tokenization longer than 512 than text skipped
            if embedded_row is not None:
                embeddings_lst.append(embedded_row)

        embeddings_tensor = torch.stack(embeddings_lst)

        return embeddings_tensor


    def cuda_mps_cpu(self):
        '''
        States the computational device in use
        '''
        if torch.cuda.is_available():  # use GPU if available
            print('Using GPU')
            return torch.device('cuda')
        # https://www.linkedin.com/pulse/how-use-gpu-tensorflow-pytorch-libraries-macbook-pro-m2apple-kashyap/
        elif torch.backends.mps.is_available():
            print('Using MPS')
            return torch.device('mps')
        else:
            print('Using CPU')
            return torch.device('cpu')


    def random_seed_function(self):
        '''
        Sets the random seed for replicability
        '''
        torch.manual_seed(self.random_seed)

        if self.device == 'cuda':
            torch.cuda.manual_seed_all(self.random_seed)

        random.seed(self.random_seed)

        return None

In [ ]:
data_class = BERT_Data()

In [ ]:
data_class.clean_bills()

In [ ]:
nyt_df = pd.read_csv(data_class.minimal_clean_nytimes_csvpath)

print(f'length = {len(nyt_df)}')

truncated_nyt_df = nyt_df.sample(n = 20000, random_state = data_class.random_seed).copy()

truncated_nyt_df

print(f'length = {len(truncated_nyt_df)}')

truncated_nyt_df.loc[:, 'nytimes_truncated_new_index'] = list(range(len(truncated_nyt_df)))

truncated_nyt_df.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/news_data/nyt_clean_minimal_truncated.csv')


In [ ]:
wapo_df = pd.read_csv(data_class.minimal_clean_wapo_csvpath)

print(f'length = {len(wapo_df)}')

truncated_wapo_df = wapo_df.sample(n = 20000, random_state = data_class.random_seed).copy()

print(f'length = {len(truncated_wapo_df)}')

truncated_wapo_df.loc[:, 'wapo_truncated_new_index'] = list(range(len(truncated_wapo_df)))

truncated_wapo_df.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/news_data/washington_post_with_date_clean_minimal_truncated.csv')


In [ ]:
df_bills = pd.read_csv(data_class.clean_bills_csvpath)
df_foxnews = pd.read_csv(data_class.minimal_clean_foxnews_csvpath)
df_breitbart = pd.read_csv(data_class.minimal_clean_breitbart_csvpath)
df_cnn = pd.read_csv(data_class.minimal_clean_cnn_csvpath)
df_nytimes =  pd.read_csv(data_class.truncated_minimal_clean_nytimes_csvpath)
df_wapo = pd.read_csv(data_class.truncated_minimal_clean_wapo_csvpath)

In [ ]:
truncated_wapo_df.head(2)

In [ ]:
index_dfcolumns_lst = ['new_index', 'bill_index', 'bill_index_2017-01', 'bill_index_2017-02', 'bill_index_2017-03', 'bill_index_2017-04', 'bill_index_2017-05', 'bill_index_2017-06', 'bill_index_2017-07', 'bill_index_2017-08', 'bill_index_2017-09', 'bill_index_2017-10', 'foxnews_index', 'foxnews_index_2016-11', 'foxnews_index_2016-12', 'foxnews_index_2017-01', 'foxnews_index_2017-02', 'foxnews_index_2017-03', 'foxnews_index_2017-04', 'foxnews_index_2017-05', 'foxnews_index_2017-06', 'foxnews_index_2017-07', 'foxnews_index_2017-08', 'foxnews_index_2017-09', 'foxnews_index_2017-10', 'foxnews_index_2017-11', 'foxnews_index_2017-12', 'breitbart_index', 'breitbart_index_2016-11', 'breitbart_index_2016-12', 'breitbart_index_2017-01', 'breitbart_index_2017-02', 'breitbart_index_2017-03', 'breitbart_index_2017-04', 'breitbart_index_2017-05', 'breitbart_index_2017-06', 'breitbart_index_2017-07', 'breitbart_index_2017-08', 'breitbart_index_2017-09', 'breitbart_index_2017-10', 'breitbart_index_2017-11', 'breitbart_index_2017-12', 'cnn_index', 'cnn_index_2016-11', 'cnn_index_2016-12', 'cnn_index_2017-01', 'cnn_index_2017-02', 'cnn_index_2017-03', 'cnn_index_2017-04', 'cnn_index_2017-05', 'cnn_index_2017-06', 'cnn_index_2017-07', 'cnn_index_2017-08', 'cnn_index_2017-09', 'cnn_index_2017-10', 'cnn_index_2017-11', 'cnn_index_2017-12', 'nytimes_index', 'nytimes_index_2016-11', 'nytimes_index_2016-12', 'nytimes_index_2017-01', 'nytimes_index_2017-02', 'nytimes_index_2017-03', 'nytimes_index_2017-04', 'nytimes_index_2017-05', 'nytimes_index_2017-06', 'nytimes_index_2017-07', 'nytimes_index_2017-08', 'nytimes_index_2017-09', 'nytimes_index_2017-10',  'nytimes_index_2017-11', 'nytimes_index_2017-12', 'wapo_index', 'wapo_index_2016-11', 'wapo_index_2016-12', 'wapo_index_2017-01', 'wapo_index_2017-02', 'wapo_index_2017-03', 'wapo_index_2017-04', 'wapo_index_2017-05', 'wapo_index_2017-06', 'wapo_index_2017-07', 'wapo_index_2017-08', 'wapo_index_2017-09', 'wapo_index_2017-10', 'wapo_index_2017-11', 'wapo_index_2017-12']

df_column_ranges = [len(pd.read_csv(data_class.minimal_clean_bills_csvpath)
        ), len(pd.read_csv(data_class.minimal_clean_foxnews_csvpath)), len(
        pd.read_csv(data_class.minimal_clean_breitbart_csvpath)), len(
        pd.read_csv(data_class.minimal_clean_cnn_csvpath)), len(
        pd.read_csv(data_class.truncated_minimal_clean_nytimes_csvpath)), len(
        pd.read_csv(data_class.truncated_minimal_clean_wapo_csvpath))]

rows_max = 20000

master_index_df = pd.DataFrame(np.full((rows_max, len(index_dfcolumns_lst)), np.nan),
    columns = index_dfcolumns_lst
)

master_index_df.loc[:, 'new_index'] = list(range(len(master_index_df)))

In [ ]:
ranges_index = 0

for index, column_name in enumerate(index_dfcolumns_lst):
    if index > 0:
        if '_index' == column_name[-6:]:
            column_index_len = df_column_ranges[ranges_index]
            nan_lst_len = rows_max - column_index_len
            master_index_df.loc[:, column_name] = list(
                    range(df_column_ranges[ranges_index])) + ([np.nan] *
                    nan_lst_len
            )
            ranges_index += 1

In [ ]:
df_bills.columns

In [ ]:
df_bills.loc[:, ['bill_date']] = df_bills.loc[:, 'date']
df_bills.loc[:, ['bill_title']] = df_bills.loc[:, 'title']
master_index_df = pd.merge(master_index_df,
        df_bills.loc[:,['bill_date', 'bill_slug', 'title', 'new_index']], on = 'new_index',
        how = 'left'
)

In [ ]:
master_index_df.columns

In [ ]:
df_foxnews.columns

In [ ]:
df_foxnews.loc[:, ['foxnews_date']] = df_foxnews.loc[:, 'date']
master_index_df = pd.merge(master_index_df,
        df_foxnews.loc[:,['foxnews_date', 'new_index']], on = 'new_index',
        how = 'left'
)

In [ ]:
master_index_df.head(2)

In [ ]:
df_breitbart.columns

In [ ]:
df_breitbart.loc[:, ['breitbart_date']] = df_breitbart.loc[:, 'date']
master_index_df = pd.merge(master_index_df,
        df_breitbart.loc[:,['breitbart_date', 'new_index']], on = 'new_index',
        how = 'left'
)

In [ ]:
df_cnn.columns

In [ ]:
df_cnn.loc[:, ['cnn_date']] = df_cnn.loc[:, 'date']
master_index_df = pd.merge(master_index_df,
        df_cnn.loc[:, ['cnn_date', 'new_index']], on = 'new_index',
        how = 'left'
)

In [ ]:
df_nytimes.columns

In [ ]:
df_nytimes.loc[:, ['nytimes_date']] = df_nytimes.loc[:, 'date']
df_nytimes.loc[:, ['nytimes_whole_new_index']] = df_nytimes.loc[:, 'new_index']
master_index_df = pd.merge(master_index_df,
        df_nytimes.loc[:,['nytimes_date', 'nytimes_truncated_new_index', 'nytimes_whole_new_index']], left_on = 'new_index',
        right_on = 'nytimes_truncated_new_index', how = 'left'
)

In [ ]:
df_wapo.columns

In [ ]:
df_wapo.loc[:, ['wapo_date']] = df_wapo.loc[:, 'date']
df_wapo.loc[:, ['wapo_whole_new_index']] = df_wapo.loc[:, 'new_index']
master_index_df = pd.merge(master_index_df,
        df_wapo.loc[:,['wapo_date', 'wapo_truncated_new_index', 'wapo_whole_new_index']], left_on = 'new_index',
        right_on = 'wapo_truncated_new_index', how = 'left'
)

In [ ]:
master_index_df.columns

In [ ]:
master_index_df

In [ ]:
master_index_df.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/bills_data/master_index_df.csv')
master_index_df.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/news_data/master_index_df.csv')

In [ ]:
master_index_df = pd.read_csv('/content/drive/Shareddrives/PulseofPolicy_data/bills_data/master_index_df.csv')

In [ ]:
master_index_df.columns

In [ ]:
suffix_lst_news = ['_index', '_date', '_2016-11', '_2016-12', '_2017-01', '_2017-02', '_2017-03', '_2017-04', '_2017-05', '_2017-06', '_2017-07', '_2017-08', '_2017-09', '_2017-10', '_2017-11', '_2017-12']

In [ ]:
suffix_lst_news[0] = '_truncated_new' + suffix_lst_news[0]

In [ ]:
master_index_df.loc[:, 'bill_date'][7]
Unnamed: 0

In [ ]:
master_index_df.columns

In [ ]:
master_index_df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
master_index_df.head(1)


In [ ]:
########################################
#      Create Master Key Dataframe     #
########################################

In [ ]:
master_index_df = pd.read_csv('/content/drive/Shareddrives/PulseofPolicy_data/bills_data/master_index_df.csv')
master_index_df.drop('Unnamed: 0', axis=1, inplace=True)
master_index_df.drop('Unnamed: 0.1', axis=1, inplace=True)
print(f'Initial master_index_df shape {master_index_df.shape}')

edit_master_index_df = master_index_df.copy()

bills_suffix_lst = ['_date', '_index_2017-01', '_index_2017-02', '_index_2017-03', '_index_2017-04', '_index_2017-05', '_index_2017-06', '_index_2017-07', '_index_2017-08', '_index_2017-09', '_index_2017-10']
news_suffix_lst = ['_date', '_index_2016-11', '_index_2016-12', '_index_2017-01', '_index_2017-02', '_index_2017-03', '_index_2017-04', '_index_2017-05', '_index_2017-06', '_index_2017-07', '_index_2017-08', '_index_2017-09', '_index_2017-10', '_index_2017-11', '_index_2017-12']


sources_lst = ['bill', 'foxnews', 'breitbart', 'cnn', 'nytimes', 'wapo']
extract_constants_lst = ['new_index']

def column_extraction(df, source, suffix_lst,
        extract_constants_lst = extract_constants_lst, truncated = False
):
    '''
    Extract columns from source dataframe based source and suffix list.
    Returns both the extracted dataframe and the columns to extract.
    '''
    df_copy = df.copy()
    # print(f'__________ini_columns_to_extract____________: {columns_to_extract}')
    columns_to_extract = []
    print(f'__________empty_columns_to_extract____________: {columns_to_extract}')
    print(f'__________extract_constants_lst____________: {extract_constants_lst}')
    columns_to_extract = extract_constants_lst
    print(f'__________with_new_index_columns_to_extract____________: {extract_constants_lst}')


    for suffix in suffix_lst:
        column_name = source + suffix

        print(f'__________column_name____________: {column_name}')
        columns_to_extract.append(column_name)
        print(f'__________columns_to_extract____________: {columns_to_extract}')

    df_copy = df_copy.loc[:, columns_to_extract]
    print(f'column_extraction function dimensions {df_copy.shape}')
    print(f'columns_to_extract: {columns_to_extract}')

    return df_copy, columns_to_extract


def merge_extracted_df(master_df, indexed_columns_df,
        columns_to_extract_lst = None
):
    '''
    Merge the extracted columns dataframe into master key dataframe.
    '''
    master_df_copy = master_df.copy()

    print(f'___________master_df_copy.columns__________{master_df_copy.columns}')

    print('before:', master_df_copy.columns)
    # print('to merge:', indexed_columns_df.head(2))
    indexed_columns_df.to_csv('/content/drive/MyDrive/UChicago/Advanced_ML/indexed_columns_df_error.csv')
    print(f'___________left_df_columns___________{indexed_columns_df.columns}')
    # if columns_to_extract_lst is not None:
    #     master_df_copy = df.drop(columns = columns_to_extract_lst)
    master_df_merged = pd.merge(master_df_copy, indexed_columns_df,
            on = 'new_index', how = 'left'
    )
    print(f'___________right_df_columns___________{master_df_copy.columns}')
    master_df_merged.to_csv('/content/drive/MyDrive/UChicago/Advanced_ML/indexed_columns_df_error_post.csv')
    # print('after:', master_df_copy)

    return master_df_merged


def index_per_month(master_df, extracted_columns_df, suffix_lst,
        columns_to_extract, source
):
    '''
    Assigns an index by whether bill or news article was written in the month in
    the extracted columns dataframe.  If it is, assigned a index, otherwise
    remains a numpy.nan.  Returns the master dataframe with the new year-month
    index.
    '''
    master_row_len = len(master_df)
    print(f'master_row_len = {master_row_len}')
    print(f'here_0 columns_to_extract: {columns_to_extract}')
    global_index = columns_to_extract[0] # new_index
    source_date = columns_to_extract[1]
    print(f'__________source_date____________: {source_date}')
    months_toindex_lst = columns_to_extract[2:]
    extracted_columns_df.loc[:, source_date] = extracted_columns_df.loc[:, source_date].str.slice(0, 7)

    for month in months_toindex_lst:
        print(f'month {month}')
        year_month = month[-7:]
        temp_df = extracted_columns_df.copy()
        temp_df = temp_df.loc[:, ['new_index', source_date]]
        print(f'__________temp_df_columns{temp_df.columns}')
        temp_df = temp_df.loc[(temp_df.loc[:, source_date] == year_month), :]
        column_index_len = len(temp_df)
        print(f'temp_row_len = {column_index_len}')
        # print(temp_df.shape, temp_df.columns)
        temp_df.loc[:, month] = list(range(len(temp_df)))
        temp_df = temp_df.loc[:, ['new_index', month]]
        # print(temp_df.shape, temp_df.columns)
        print(f"Attempting to drop: {month}")
        print(f"Columns before dropping: {master_df.columns}")
        if month in master_df.columns:
            print('MONTH IS IN COLUMNS!!!!!')
            master_df.drop([month], axis='columns', inplace=True)
            print(f"Columns after dropping: {master_df.columns}")
        else:
            print(f"Column {month} not found in DataFrame.")
        # print(f'__________master_df_columns{master_df.columns}')
        master_df = merge_extracted_df(master_df, temp_df, None)

    return master_df


def all_indexby_month(master_df, sources_lst, bills_suffix_lst, news_suffix_lst,
        extract_constants_lst, truncated = False):
    '''
    Runs the above functions on the data sources of interest, and returns the
    master dataframe with the index per month.
    '''
    df_to_edit = master_df.copy()
    new_master_df = master_df.copy()

    for source in sources_lst:
        print(f'source: {source}')
        if source == 'bill':
            suffix_lst = bills_suffix_lst
        else:
            suffix_lst = news_suffix_lst
        extracted_columns_df, columns_to_extract = column_extraction(df_to_edit,
                source, suffix_lst, ['new_index'], truncated
        )
        print(f'here_1: {columns_to_extract}')
        print(f'head')
        print(extracted_columns_df.columns)
        new_master_df = index_per_month(new_master_df, extracted_columns_df,
                suffix_lst, columns_to_extract, source
        )

    return new_master_df

In [ ]:
new_master_index_df = all_indexby_month(master_index_df, sources_lst,
        bills_suffix_lst, news_suffix_lst, extract_constants_lst, False)

In [ ]:
def df_indexconlumns_to_int(df):
    '''
    Takes a dataframe as an input and returns the same dataframe
    where all index columns are integers and not floats.
    '''
    for col in df.columns:
        if 'index' in col:
            df.loc[:, col] = df.loc[:, col].astype('Int64')
    return df

In [ ]:
new_master_index_df = df_indexconlumns_to_int(new_master_index_df)

In [ ]:
new_master_index_df.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/bills_data/v2_master_index_df.csv')

In [ ]:
new_master_index_df.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/bills_data/v2_master_index_df.csv')

new_master_index_df.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/news_data/v2_master_index_df.csv')